# Test scraper La Voz del Sur

In [8]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import random
from urllib.parse import urljoin
from datetime import datetime

# Configuración
BASE_URL = "https://letrafria.com/"
CATEGORY_URL = "https://letrafria.com/category/region/"
OUTPUT_FILE = f"letra_fria_completo_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
DELAY = random.uniform(2, 4)
MAX_PAGES = 5  # Ajustar según necesidad

# Headers optimizados
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'es-MX,es;q=0.9'
}

def get_soup(url):
    """Obtiene el HTML con manejo robusto de errores"""
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        return BeautifulSoup(response.text, 'html.parser')
    except Exception as e:
        print(f"Error al obtener {url}: {str(e)}")
        return None

def clean_text(text):
    """Limpia el texto preservando estructura"""
    if not text:
        return ""
    return ' '.join(text.strip().split())

def extract_article_data(article_url):
    """Extrae datos de artículo con selectores optimizados"""
    print(f"\nExtrayendo: {article_url}")
    soup = get_soup(article_url)
    if not soup:
        return None
    
    try:
        # 1. Extraer título - selector confirmado
        title = clean_text(soup.find('h1', class_='post-title').get_text())
        
        # 2. Extraer fecha - selectores específicos para Letra Fría
        date_elem = soup.find('span', class_='post-date')  # Selector principal
        if not date_elem:
            date_elem = soup.find('time', class_='published')  # Alternativa
            
        date = clean_text(date_elem.get_text()) if date_elem else ""
        
        # 3. Extraer autor - selector específico confirmado
        author_elem = soup.find('span', class_='post-author') or soup.find('a', class_='author-name')
        author = clean_text(author_elem.get_text()) if author_elem else ""
        
        # 4. Extraer contenido - enfoque mejorado
        content_div = soup.find('div', class_='post-content')
        paragraphs = []
        
        if content_div:
            # Eliminar elementos específicos no deseados
            unwanted = ['sharedaddy', 'code-block', 'twitter-tweet', 'wp-block-embed']
            for element in content_div.find_all(['script', 'style', 'iframe'] + unwanted):
                element.decompose()
            
            for p in content_div.find_all(['p', 'h2', 'h3']):
                text = clean_text(p.get_text())
                if text and len(text.split()) > 2:
                    paragraphs.append(text)
        
        content = '\n\n'.join(paragraphs) if paragraphs else "Contenido no disponible"
        
        print(f"✓ Título: {title}")
        print(f"✓ Fecha: {date}")
        print(f"✓ Autor: {author}")
        print(f"✓ Caracteres: {len(content)}")
        
        return {
            'titulo': title,
            'articulo': content,
            'fecha': date,
            'autor': author,
            'url': article_url,
            'categoria': 'Región'
        }
        
    except Exception as e:
        print(f"Error procesando artículo: {str(e)}")
        return None

def get_article_links(page_url):
    """Obtiene enlaces con selectores robustos"""
    print(f"\nObteniendo artículos de: {page_url}")
    soup = get_soup(page_url)
    if not soup:
        return []
    
    links = []
    # Selector principal confirmado
    articles = soup.select('article.item-post h2 a[href]')
    
    for article in articles:
        link = urljoin(BASE_URL, article['href'])
        if link not in links:
            links.append(link)
    
    print(f"Encontrados {len(links)} artículos")
    return links

def main():
    """Función principal con paginación mejorada"""
    with open(OUTPUT_FILE, mode='w', encoding='utf-8', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['titulo', 'articulo', 'fecha', 'autor', 'url', 'categoria'])
        writer.writeheader()
        
        page_num = 1
        articles_scraped = 0
        
        while page_num <= MAX_PAGES:
            page_url = CATEGORY_URL if page_num == 1 else f"{CATEGORY_URL}page/{page_num}/"
            
            article_links = get_article_links(page_url)
            
            if not article_links:
                print("No se encontraron artículos. Terminando...")
                break
            
            for link in article_links:
                article_data = extract_article_data(link)
                if article_data:
                    writer.writerow(article_data)
                    csvfile.flush()
                    articles_scraped += 1
                
                time.sleep(DELAY)
            
            page_num += 1
            time.sleep(DELAY * 1.5)
        
        print(f"\nScraping completado. Artículos extraídos: {articles_scraped}")
        print(f"Archivo guardado: {OUTPUT_FILE}")

if __name__ == "__main__":
    print("""
    ====================================
    SCRAPER LETRA FRÍA - VERSIÓN FINAL
    Extracción completa garantizada
    ====================================
    """)
    
    start_time = time.time()
    main()
    duration = time.time() - start_time
    print(f"\nTiempo total: {duration//60:.0f}m {duration%60:.0f}s")


    SCRAPER LETRA FRÍA - VERSIÓN FINAL
    Extracción completa garantizada
    

Obteniendo artículos de: https://letrafria.com/category/region/
Encontrados 0 artículos
No se encontraron artículos. Terminando...

Scraping completado. Artículos extraídos: 0
Archivo guardado: letra_fria_completo_20250414_120720.csv

Tiempo total: 0m 1s
